
# 4.3.3 htmファイルからMD＆A情報としてみなされる文章のみを抽出 

　htmファイルから、 BeautifulSoupを用いて MD＆A情報としてみなされる文章のみを抽出した。BeautifulSoupとは、Pythonのライブラリのひとつで、htmlやxmlといったWebページで用いられるデータを構文解析する際に用いられるライブラリである。今回は、 htm形式のテキストファイルからMD&A部分を抜き出し、htm形式のタグの除去等の処理を行って生の文章を抽出した。また文章を抽出においては、加藤・五島(2020)を参考に「１【経営方針、経営環境及び対処すべき課題等】」と「３【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】」の 2 つの章のテキストをMD&Aの記述とみなして抽出を行った。

In [5]:
from datetime import datetime as dt
import glob
import pandas as pd
import bs4
import re
from bs4 import BeautifulSoup
import os


def fetch_md_and_a(data_frame_name, data_frame):

    htm_files = call_htm_files(data_frame_name)
    industry_htm_files_hash = make_industry_htm_files_hash(htm_files , data_frame_name, data_frame)
    
    sum_csv_output = 1
    for industry_name, htm_files in industry_htm_files_hash.items():
        
        edinet_id_files_hash = {}
        #Edninetの識別子ごとでファイル群を管理する
        for file in htm_files:
            pattern = 'E.*?-'
            result = re.search(pattern, file)
            start, end = result.span()
            identifier = file[start: end-1]

            if identifier in edinet_id_files_hash :
                edinet_id_files_hash[identifier].append(file)
            else:
                edinet_id_files_hash[identifier] = [file]
        
        idx_num = 1
        # すでにEdninetの識別子順でソートされている
        for identifier, e_files in edinet_id_files_hash.items() :
            # 「１【経営方針、経営環境及び対処すべき課題等】」,「３【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】」            
            #コロナ前, コロナ過渡期, コロナ後の最低３つがなかった時
#             if len(e_files) < 3:
#                 print(f"detected: {e_files}")
#                 continue
            
            date_files_hash = {}
            for e_f in e_files:
                date_str = get_date_str_val(e_f)
                if date_str in date_files_hash :
                    date_files_hash[date_str].append(e_f)
                else:
                    date_files_hash[date_str] = [e_f]
            
            for date_str, d_files in date_files_hash.items():
                
                d_files_len = len(d_files)

                if d_files_len > 1:
                    print(f"count: {d_files_len}, {d_files}")
                
                md_a_df = pd.DataFrame()
                idx = 0
                while  idx < d_files_len :
                    sp = read_file(d_files[idx])
                    htm_contents = sp.find('body').div.contents
                    md_a_contents, func =  extract_md_a_contents(htm_contents)
                    tmp_df = func(md_a_contents)                
                    md_a_df  = pd.concat([md_a_df, tmp_df], axis=0)
                    idx += 1

                save_md_a_df_to_csv(md_a_df, idx_num, date_str , data_frame_name,  industry_name)
                #beforeとafterでユニークなindex_numを持つようにする
                idx_num += 1
                
                print(sum_csv_output, len(md_a_df))
                sum_csv_output += 1

In [6]:
def call_htm_files(data_frame_name):
    htm_files = glob.glob(f"/home/jovyan/work/2UnzippingHtm/UnzipedHtmFiles/{data_frame_name}/**/*.htm", recursive=True)
    return  htm_files


def make_industry_htm_files_hash(htm_files , data_frame_name: str, data_frame):
    industry_htm_files_hash = {}
    
    industry_list = make_type_of_industry_list(data_frame)
    for industry in industry_list:
        industry_htm_files_hash[industry]= list(filter(lambda x:  industry in x , htm_files))
    
    return  industry_htm_files_hash 


def make_type_of_industry_list(data_frame : pd.DataFrame, industry_col="[業種（東証）]"):
    return ["サービス業"]


def get_date_str_val(htm_file) :
    date_str = htm_file[-20:-10]
    return  date_str


def read_file(htm_path: str) -> BeautifulSoup:
    with open(htm_path, mode="r") as f:
        htm = f.read()
        sp = BeautifulSoup(htm, "html.parser")
        return sp

"""
htmファイルには3つのパターンが存在しているため、その都度ハンドリングをする


1. smt_textパターン: 属性値にsmt_textを持っているhtm
2. textjustify_spanパターン: smt_textを持たず、spanにテキスト情報を記載しているhtm
3. text_indent?
"""
#-----------------------------------------------------
#md_aの章番号を指定
#企業によって全角と半角かわかれる可能性があるためどちらも判定
# 「１【経営方針、経営環境及び対処すべき課題等】」,「３【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】」
# after_2017 = ['１', '1','３', '3']

def extract_md_a_contents(htm_contents: list, chapter_numbers=['１', '1','３', '3']) -> tuple:
    
    #MD＆A情報を持つ章を抽出
    md_a_contents = []
    
    for content in htm_contents :
        #bs4.element.Tag型ではないとfindメソッドでtypeエラーを起こすため判定
        if not isinstance(content, bs4.element.Tag) :
            continue
        
        found = content.find(class_=re.compile("smt_head"))
        #findしたけどデータがない(s4.element.Tag型ではない)がない場合.textプロパティを呼び出す際にtypeエラーを起こすため判定
        if not  found:
            found = content.find("h3")
            if not  found:
                continue
            else: 
                func = insert_df_by_span_p
        else :
            func = insert_df_by_smt
    
        title = found.text
        #md_aの章番号を指定
            # "1【経営方針、経営環境及び対処すべき課題等】",
            # "3【経営者による財政状態、経営成績及びキャッシュ・フローの状況の分析】"
        #企業によって全角と半角か別れる可能性があるためどちらも判定
        if title[0] in  chapter_numbers :
             md_a_contents.append(content)
            
    return md_a_contents, func

#------------------------------------------
def insert_df_by_span_p(md_a_contents: list) -> pd.DataFrame:
    
    md_a_df = pd.DataFrame()

    text_values = []
    for content in  md_a_contents : 
        
        # テキスト情報を持つbs4.element.Tag型のみ取得
        found_values = content.find_all("p")
         # テキストを取得
      
        for val in found_values:
            if val.parent.name in ["td" , "tr"] or val.parent.parent.name in ["td" , "tr"]:
                continue
    
            if val.span != None:
                text = val.span.text
                if len(text) > 1:
                    text_values.append(text)
            else :
                text = val.text
                if len(text) > 1:
                    text_values.append(text)
     
        tmp_df = pd.DataFrame()
        tmp_df['Text'] = text_values 
        #pandas.DataFrame型に入れる
        md_a_df  = pd.concat([md_a_df ,  tmp_df], axis=0)
        
    return md_a_df

#-----------------------------------------------------
def insert_df_by_smt(md_a_contents: list) -> pd.DataFrame:
    
    md_a_df = pd.DataFrame()

    for content in  md_a_contents : 
        # テキスト情報を持つbs4.element.Tag型のみ取得
        found_values = content.find_all(class_=re.compile("smt_text"))
         # テキストを取得
        text_values = list(filter(lambda val: len(val) > 1, list(map(lambda val: val.text,found_values))))
        tmp_df = pd.DataFrame()
        tmp_df['Text'] = text_values 
        #pandas.DataFrame型に入れる
        md_a_df  = pd.concat([md_a_df ,  tmp_df], axis=0)
        
    return md_a_df

#----------------------------------------------
        
def save_md_a_df_to_csv(md_a_df: pd.DataFrame, idx_num: int, date_str: str,  data_frame_name, industry_name) -> None :
    #Sampleフォルダの作成
    
    
    filepath = os.getcwd()+ f"/{data_frame_name}"
    if not os.path.exists(filepath) :
        os.mkdir(filepath)
        
    filepath = os.getcwd()+ f"/{data_frame_name}/{industry_name}"
    if not os.path.exists(filepath) :
        os.mkdir(filepath)
    
    filepath_before_pandemic  = os.getcwd()+ f"/{data_frame_name}/{industry_name}"  +  "/BeforeSample"
    if not os.path.exists(filepath_before_pandemic) :
        os.mkdir(filepath_before_pandemic)
    
    filepath_transition_period_pandemic = os.getcwd()+ f"/{data_frame_name}/{industry_name}"  +  "/TransitionPeriodSample"
    if not os.path.exists(filepath_transition_period_pandemic) :
        os.mkdir(filepath_transition_period_pandemic)
    
    filepath_after_pandemic = os.getcwd()+ f"/{data_frame_name}/{industry_name}"  +  "/AfterSample"
    if not os.path.exists(filepath_after_pandemic) :
        os.mkdir(filepath_after_pandemic)
            
    
    date_dt = dt.strptime(date_str, '%Y-%m-%d')
    
    before_pandemic_boundary_val = dt(2019, 3, 31)
    
    transition_period_pandemic_boundary_val = dt(2020, 3, 31)
    if  date_dt <= before_pandemic_boundary_val :
        md_a_df.to_csv(filepath_before_pandemic+"/"+f'{idx_num}_{date_str}.csv')
    elif  date_dt <= transition_period_pandemic_boundary_val :
        md_a_df.to_csv(filepath_transition_period_pandemic +"/"+f'{idx_num}_{date_str}.csv')
    else :
        md_a_df.to_csv(filepath_after_pandemic+"/"+f'{idx_num}_{date_str}.csv')

In [7]:
import pandas as pd

# data_frame_name = "ave_top_20"
data_frame_name = "ave_worst_20"
# data_frame_name = "2021_top_20"
# data_frame_name = "2021_worst_20"
data_frame = pd.read_csv("/home/jovyan/work/1CallingEdinetApi"+f"/EdinetIdxFiles/{data_frame_name}.csv", skiprows=4)

In [8]:
fetch_md_and_a(data_frame_name, data_frame)

1 142
2 138
3 150
4 47
5 93
6 79
7 84
8 74
9 72
10 62
11 61
12 64
13 120
14 127
15 89
16 178
17 185
18 84
19 123
20 102
21 115
22 39
23 59
24 49
25 42
26 52
27 88
28 32
29 27
30 26
31 181
32 154
33 127
34 97
35 138
36 107
37 133
38 155
39 144
40 129
41 136
42 151
43 96
44 119
45 109
46 131
47 144
48 160
49 165
50 171
51 169
52 164
53 148
54 114
55 96
56 94
57 99
58 57
59 89
60 118
61 40
62 47
63 61
64 164
65 174
66 162
67 63
68 75
69 53
70 115
71 120
72 128
73 111
74 56
75 48
76 133
77 133
78 155
79 58
80 50
81 52
82 65
83 70
84 89
85 96
86 95
87 98
88 42
89 44
90 82
91 185
92 177
93 164
94 57
95 49
96 69
97 123
98 114
99 116
100 60
101 60
102 73
103 153
104 172
105 174
106 52
107 64
108 84
109 106
110 170
111 173
